## Introduction

Provide a brief introduction to the purpose of this notebook.

## Import libraries

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style='whitegrid')
import missingno as msno

## Load the data

The NOAA National Centers for Environmental Information provides access to many types of environmental data, including records of daily precipitation.

We can use their [website](https://www.ncei.noaa.gov/cdo-web/search?datasetid=GHCND) to request records of daily precipitation from Seattle and St. Louis (or other locations of interest) for the last 5 years (2018 - 2022). 

I have already obtained the data. The data are available at this [github repository](https://github.com/brian-fischer/DATA-3320/tree/main/weather) and are called `seattle_rain.csv` and `stl_rain.csv`.

Below, we will load and clean the datasets from github and then merge them to create a dataset for the analysis steps of the data science methodology to answer the question of whether it rains more in Seattle, WA than in St. Louis, MO.

Load the Seattle data set

In [7]:
df_seattle = pd.read_csv('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/weather/seattle_rain.csv')

Load the St. Louis data set

In [8]:
df_stlouis = pd.read_csv('https://raw.githubusercontent.com/brian-fischer/DATA-3320/main/weather/stl_rain.csv')

## Explore the contents of the data sets

Display the beginning of the SEA dataset

In [9]:
df_seattle.head()

,STATION,NAME,DATE,DAPR,MDPR,PRCP,SNOW,SNWD,WESD,WESF
0,US1WAKG0225,"SEATTLE 2.1 ESE, WA US",1/1/18,NaN,NaN,0.00,NaN,NaN,NaN,NaN
1,US1WAKG0225,"SEATTLE 2.1 ESE, WA US",1/2/18,NaN,NaN,0.00,NaN,NaN,NaN,NaN
2,US1WAKG0225,"SEATTLE 2.1 ESE, WA US",1/3/18,NaN,NaN,0.00,NaN,NaN,NaN,NaN
3,US1WAKG0225,"SEATTLE 2.1 ESE, WA US",1/4/18,NaN,NaN,0.00,NaN,NaN,NaN,NaN
4,US1WAKG0225,"SEATTLE 2.1 ESE, WA US",1/5/18,NaN,NaN,0.25,NaN,NaN,NaN,NaN


Display the beginning of the STL dataset

In [10]:
df_stlouis.head()

,STATION,NAME,DATE,DAPR,MDPR,PRCP,SNOW,SNWD
0,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2017-01-01,NaN,NaN,0.00,0.0,NaN
1,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2017-01-03,NaN,NaN,0.35,NaN,NaN
2,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2017-01-04,NaN,NaN,0.03,NaN,NaN
3,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2017-01-05,NaN,NaN,0.04,1.2,NaN
4,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2017-01-06,NaN,NaN,0.00,0.0,1.0


## Convert data types, if necessary

Display columns, number of entries, and data types with the indexes using pandas .info()

In [11]:
df_seattle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1658 entries, 0 to 1657
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  1658 non-null   object 
 1   NAME     1658 non-null   object 
 2   DATE     1658 non-null   object 
 3   DAPR     23 non-null     float64
 4   MDPR     23 non-null     float64
 5   PRCP     1636 non-null   float64
 6   SNOW     353 non-null    float64
 7   SNWD     66 non-null     float64
 8   WESD     15 non-null     float64
 9   WESF     28 non-null     float64
dtypes: float64(7), object(3)
memory usage: 129.7+ KB


In [12]:
df_stlouis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54574 entries, 0 to 54573
Data columns (total 8 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   STATION  54574 non-null  object 
 1   NAME     54574 non-null  object 
 2   DATE     54574 non-null  object 
 3   DAPR     1166 non-null   float64
 4   MDPR     1163 non-null   float64
 5   PRCP     53143 non-null  float64
 6   SNOW     33167 non-null  float64
 7   SNWD     12835 non-null  float64
dtypes: float64(5), object(3)
memory usage: 3.3+ MB


The Dates in both datasets will need to be converted later for analysis

Pull all unique station codes from the STL dataframe, since all the SEA data comes from only one station

In [13]:
df_stlouis['STATION'].unique()

array(['US1MOSS0027', 'US1MOSL0019', 'US1MOSL0018', 'US1MOSL0039',
       'US1MOSLC010', 'US1ILSC0009', 'US1MOSL0050', 'US1MOJF0018',
       'US1MOSL0070', 'US1MOSL0092', 'US1MOSL0079', 'US1MOSL0056',
       'US1MOSL0055', 'US1MOSL0077', 'US1ILMD0047', 'US1MOSL0054',
       'US1MOSL0076', 'US1MOSL0074', 'US1MOSLC007', 'US1MOFSA189',
       'US1MOSL0027', 'US1MOSL0049', 'US1MOSL0004', 'USC00237397',
       'USC00237452', 'USC00237398', 'US1ILSC0018', 'US1MOSS0034',
       'USW00003960', 'US1MOSL0083', 'US1ILMO0006', 'US1MOSL0081',
       'US1MOJF0025', 'US1MOSS0051', 'US1ILMD0010', 'US1MOSL0024',
       'US1MOSL0045', 'US1MOSL0067', 'US1MOSL0020', 'US1ILMD0012',
       'US1MOSL0063', 'USW00013994', 'US1MOSL0062', 'US1MOSL0084'],
      dtype=object)

Count the number of observations recorded for each station

In [14]:
df_seattle['STATION'].value_counts()

US1WAKG0225    1658
Name: STATION, dtype: int64

In [15]:
df_stlouis['STATION'].value_counts()

USW00003960    2191
USW00013994    2191
USC00237397    2189
USC00237398    2189
US1MOSLC007    2179
USC00237452    2154
US1ILMD0010    2150
US1MOSL0024    2122
US1MOJF0018    2113
US1MOSL0027    2085
US1ILMD0012    1954
US1MOSL0049    1938
US1MOSL0018    1923
US1MOSL0004    1868
US1MOSL0050    1810
US1MOSL0055    1792
US1MOFSA189    1776
US1MOSL0074    1674
US1MOSS0027    1663
US1MOSL0020    1544
US1MOSL0067    1473
US1ILSC0009    1399
US1MOSL0039    1354
US1MOSL0077    1339
US1MOJF0025    1124
US1MOSL0070     965
US1MOSL0063     924
US1MOSL0081     914
US1MOSL0045     849
US1MOSL0062     807
US1MOSL0083     646
US1ILSC0018     453
US1MOSL0084     401
US1MOSL0079     371
US1MOSS0051     302
US1MOSLC010     291
US1MOSL0054     248
US1MOSL0092     232
US1ILMD0047     218
US1MOSL0076     211
US1MOSL0056     198
US1ILMO0006     162
US1MOSS0034     130
US1MOSL0019      58
Name: STATION, dtype: int64

# Convert Date to DateTime

Convert the date columns for both datasets to DATETIME

In [16]:
df_seattle['DATE'] = pd.to_datetime(df_seattle['DATE'])

In [17]:
df_stlouis['DATE'] = pd.to_datetime(df_stlouis['DATE'])

## Select relevant subsets of the data

Subset the St. Louis dataframe to include only data from 2018 and beyond, since the Seattle data only includes 2018 and beyond

In [18]:
df_stlouis2018 = df_stlouis.loc[df_stlouis['DATE'] >= '2018-01-01'] ##.loc (explicit) is automatically run, even when not included in the code
df_stlouis2018.head()

,STATION,NAME,DATE,DAPR,MDPR,PRCP,SNOW,SNWD
241,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2018-01-05,NaN,NaN,0.00,0.2,1.0
242,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2018-01-08,NaN,NaN,0.24,NaN,NaN
243,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2018-01-10,NaN,NaN,0.00,NaN,NaN
244,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2018-01-11,NaN,NaN,0.02,NaN,NaN
245,US1MOSS0027,"ST. CHARLES 2.3 NE, MO US",2018-01-15,NaN,NaN,0.05,0.5,NaN


select subset for one station in St. Louis, since the Seattle data only comes from one station

In [19]:
df_stlouisLAMBERT = df_stlouis2018.loc[df_stlouis2018['NAME'] == 'ST LOUIS LAMBERT INTERNATIONAL AIRPORT, MO US']
df_stlouisLAMBERT['STATION'].value_counts()

USW00013994    1826
Name: STATION, dtype: int64

## Join data frames keeping `DATE` and `PRCP` columns

Left merge the STL dataset on DATE column of SEA dataset, selecting only the DATE and PRECIPITATION columns (since that's all that we need for the analysis)

In [20]:
df_both = df_stlouisLAMBERT[['DATE', 'PRCP']].merge(df_seattle[['DATE', 'PRCP']], on ='DATE', how = 'left')

df_both

,DATE,PRCP_x,PRCP_y
0,2018-01-01,0.00,0.00
1,2018-01-02,0.00,0.00
2,2018-01-03,0.00,0.00
3,2018-01-04,0.00,0.00
4,2018-01-05,0.00,0.25
...,...,...,...
1821,2022-12-27,0.00,0.78
1822,2022-12-28,0.00,0.40
1823,2022-12-29,0.00,0.03
1824,2022-12-30,0.31,0.62


## Create a tidy data frame with columns for city and precipitation

create a tidy format by using pandas .melt() to transform the new combined dataset into a long format instead of wide.

In [21]:
df_both_tidy = pd.melt(df_both, id_vars = 'DATE', var_name = 'CITY', value_name = 'PRCP')

df_both_tidy

,DATE,CITY,PRCP
0,2018-01-01,PRCP_x,0.00
1,2018-01-02,PRCP_x,0.00
2,2018-01-03,PRCP_x,0.00
3,2018-01-04,PRCP_x,0.00
4,2018-01-05,PRCP_x,0.00
...,...,...,...
3647,2022-12-27,PRCP_y,0.78
3648,2022-12-28,PRCP_y,0.40
3649,2022-12-29,PRCP_y,0.03
3650,2022-12-30,PRCP_y,0.62


rename the "new columns" to indicate the city names so that the column names are relevant

In [22]:
df_both_tidy.loc[df_both_tidy['CITY'] == 'PRCP_x', 'CITY'] = 'STL'
df_both_tidy.loc[df_both_tidy['CITY'] == 'PRCP_y', 'CITY'] = 'SEA'

df_both_tidy

,DATE,CITY,PRCP
0,2018-01-01,STL,0.00
1,2018-01-02,STL,0.00
2,2018-01-03,STL,0.00
3,2018-01-04,STL,0.00
4,2018-01-05,STL,0.00
...,...,...,...
3647,2022-12-27,SEA,0.78
3648,2022-12-28,SEA,0.40
3649,2022-12-29,SEA,0.03
3650,2022-12-30,SEA,0.62


rename all columns to lowercase

In [23]:
df_rain = df_both_tidy.rename(columns = {'DATE' : 'date', 'CITY' : 'city', 'PRCP' : 'precipitation'})

df_rain

,date,city,precipitation
0,2018-01-01,STL,0.00
1,2018-01-02,STL,0.00
2,2018-01-03,STL,0.00
3,2018-01-04,STL,0.00
4,2018-01-05,STL,0.00
...,...,...,...
3647,2022-12-27,SEA,0.78
3648,2022-12-28,SEA,0.40
3649,2022-12-29,SEA,0.03
3650,2022-12-30,SEA,0.62


## Create relevant derived variables as new columns

Create day of year column ('day_of_year') that counts each date's observation as the number of days away it is from the 1st day of the year.

In [24]:
df_rain['day_of_year'] = pd.DatetimeIndex(df_rain['date']).day_of_year

df_rain

,date,city,precipitation,day_of_year
0,2018-01-01,STL,0.00,1
1,2018-01-02,STL,0.00,2
2,2018-01-03,STL,0.00,3
3,2018-01-04,STL,0.00,4
4,2018-01-05,STL,0.00,5
...,...,...,...,...
3647,2022-12-27,SEA,0.78,361
3648,2022-12-28,SEA,0.40,362
3649,2022-12-29,SEA,0.03,363
3650,2022-12-30,SEA,0.62,364


Add a 'month' column to the combined dataframe that lists each month number (1-12) for each observation

In [25]:
df_rain['month'] = pd.DatetimeIndex(df_rain['date']).month

df_rain

,date,city,precipitation,day_of_year,month
0,2018-01-01,STL,0.00,1,1
1,2018-01-02,STL,0.00,2,1
2,2018-01-03,STL,0.00,3,1
3,2018-01-04,STL,0.00,4,1
4,2018-01-05,STL,0.00,5,1
...,...,...,...,...,...
3647,2022-12-27,SEA,0.78,361,12
3648,2022-12-28,SEA,0.40,362,12
3649,2022-12-29,SEA,0.03,363,12
3650,2022-12-30,SEA,0.62,364,12


## Impute missing values

Check for missing values in the combined dataset

In [26]:
df_rain[df_rain['precipitation'].isna()]

,date,city,precipitation,day_of_year,month
1834,2018-01-09,SEA,NaN,9,1
1835,2018-01-10,SEA,NaN,10,1
1836,2018-01-11,SEA,NaN,11,1
1837,2018-01-12,SEA,NaN,12,1
1838,2018-01-13,SEA,NaN,13,1
...,...,...,...,...,...
3368,2022-03-23,SEA,NaN,82,3
3369,2022-03-24,SEA,NaN,83,3
3370,2022-03-25,SEA,NaN,84,3
3371,2022-03-26,SEA,NaN,85,3


Counts the entries with missing values (sum)

In [27]:
df_rain.isna().sum()

date               0
city               0
precipitation    190
day_of_year        0
month              0
dtype: int64

get the index for each row with 'precipitation' value missing

In [28]:
indices = np.where(df_rain['precipitation'].isna() == True)[0] ## [0] reduces to single dimension array of all missing

indices

array([1834, 1835, 1836, 1837, 1838, 1839, 1840, 1841, 1842, 1843, 1844,
       1845, 1846, 1847, 1848, 1849, 1850, 1851, 1852, 1853, 1854, 1855,
       1856, 1857, 1858, 1859, 1860, 1861, 1862, 1863, 1864, 1865, 1866,
       1867, 1868, 1869, 1870, 1871, 1872, 1873, 1874, 1875, 1876, 1877,
       1878, 1879, 1880, 1881, 1882, 1883, 1884, 1885, 1886, 1887, 1888,
       1889, 1890, 1891, 1892, 1893, 1894, 1895, 2090, 2131, 2132, 2133,
       2134, 2135, 2136, 2137, 2138, 2139, 2140, 2195, 2196, 2197, 2214,
       2215, 2244, 2245, 2246, 2247, 2248, 2249, 2286, 2287, 2288, 2362,
       2363, 2368, 2369, 2370, 2371, 2372, 2373, 2374, 2375, 2376, 2377,
       2417, 2418, 2419, 2420, 2421, 2422, 2423, 2517, 2518, 2519, 2520,
       2521, 2522, 2523, 2524, 2559, 2560, 2561, 2602, 2603, 2604, 2605,
       2606, 2607, 2608, 2609, 2610, 2611, 2612, 2818, 2819, 2820, 2821,
       2822, 2823, 2824, 2825, 2826, 2827, 2972, 2973, 2974, 2975, 2983,
       2984, 2986, 2987, 2988, 3000, 3001, 3004, 30

Run these two lines of code if you would like to check for leap years:

In [29]:
## pd.DatetimeIndex(df_rain.loc[df_rain['date'] == '2019-03-01', 'date']).day_of_year 

##checking for leap year in 2019

In [30]:
## pd.DatetimeIndex(df_rain.loc[df_rain['date'] == '2020-03-01', 'date']).day_of_year 

## 2020 has a leap day

compute mean precipitation for each day in seattle, avg across years

In [31]:
mean_day_prcp = df_rain.loc[df_rain['city'] == 'SEA', ['precipitation', 'day_of_year']].groupby('day_of_year').mean()
mean_day_prcp

,precipitation
day_of_year,
1,0.052000
2,0.150000
3,0.836000
4,0.370000
5,0.246667
...,...
362,0.120000
363,0.102000
364,0.268000


for each missing val, replace it with mean daily precipitation value that was calculated ('mean_day_prcp')

In [32]:
for _, index in enumerate(indices):
  df_rain.loc[index, 'precipitation'] = mean_day_prcp.loc[df_rain.loc[index, 'day_of_year']].values[0]

Check again for missing values (there should be no results now)

In [33]:
df_rain[df_rain['precipitation'].isna() == True]

,date,city,precipitation,day_of_year,month


# Export the clean .csv file

In [35]:
from google.colab import files

df_rain.to_csv('clean_seattle_stl_weather.csv', encoding = 'utf-8-sig', index=False) 

files.download('clean_seattle_stl_weather.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>